In [2]:
import os
import uuid

from datetime import datetime, timedelta
import values, helpers
import random
import pandas as pd
import numpy as np
import create_sap_table.create_table_leanx as sap_table

In [3]:
# required tables
VBAK = pd.DataFrame(columns=[col[0] for col in sap_table.fetch_table(table_name='VBAK')])
VBAP = pd.DataFrame(columns=[col[0] for col in sap_table.fetch_table(table_name='VBAP')])

LIKP = pd.DataFrame(columns=[col[0] for col in sap_table.fetch_table(table_name='LIKP')])
LIPS = pd.DataFrame(columns=[col[0] for col in sap_table.fetch_table(table_name='LIPS')])

MKPF = pd.DataFrame(columns=[col[0] for col in sap_table.fetch_table(table_name='MKPF')])
MSEG = pd.DataFrame(columns=[col[0] for col in sap_table.fetch_table(table_name='MSEG')])

BKPF = pd.DataFrame(columns=[col[0] for col in sap_table.fetch_table(table_name='BKPF')])
BSEG = pd.DataFrame(columns=[col[0] for col in sap_table.fetch_table(table_name='BSEG')])

VBRK = pd.DataFrame(columns=[col[0] for col in sap_table.fetch_table(table_name='VBRK')])
VBRP = pd.DataFrame(columns=[col[0] for col in sap_table.fetch_table(table_name='VBRP')])

CDHDR = pd.DataFrame(columns=[col[0] for col in sap_table.fetch_table(table_name='CDHDR')])
CDPOS = pd.DataFrame(columns=[col[0] for col in sap_table.fetch_table(table_name='CDPOS')])

USR02 = pd.DataFrame(columns=[col[0] for col in sap_table.fetch_table(table_name='USR02')])
MARA = pd.DataFrame(columns=[col[0] for col in sap_table.fetch_table(table_name='MARA')])
KNA1 = pd.DataFrame(columns=[col[0] for col in sap_table.fetch_table(table_name='KNA1')])

In [4]:
class User:
    def __init__(self, bname, ustyp, mandt=values.mandt) -> None:
        self.mandt = mandt
        self.bname = bname
        self.ustyp = ustyp

class Material:
    def __init__(self, matnr, price, mandt=values.mandt) -> None:
        self.mandt = mandt
        self.matnr = matnr
        self.price = price

class Customer:
    def __init__(self, kunnr, erdat, mandt=values.mandt) -> None:
        self.mandt = mandt
        self.kunnr = kunnr
        self.erdat = erdat
        
class SalesOrderItem:
    def __init__(self, vbeln, posnr, mandt=values.mandt) -> None:
        self.mandt = mandt
        self.vbeln = vbeln
        self.posnr = posnr

In [5]:
# insert tables and create objects list
USERS = []
for k, v in values.users.items():
    users_last_index = len(USR02)
    USR02.loc[users_last_index, 'MANDT'] = values.mandt
    USR02.loc[users_last_index, 'BNAME'] = k
    USR02.loc[users_last_index, 'USTYP'] = v

    USERS.append(User(bname=k, ustyp=v))

CUSTOMERS = []
for k, v in values.customers.items():
    kna1_last_index = len(KNA1)
    rand_kunnr = uuid.uuid4()
    rand_erdat = helpers.generate_random_datetime(start_date=datetime(2009, 1, 1), end_date=datetime(2010, 1, 1))

    KNA1.loc[kna1_last_index, 'MANDT'] = values.mandt
    KNA1.loc[kna1_last_index, 'KUNNR'] = rand_kunnr

    CUSTOMERS.append(Customer(kunnr=rand_kunnr, erdat=rand_erdat))

MATERIALS = []
for k, v in values.materials.items():
    mara_last_index = len(MATERIALS)
    rand_matnr = uuid.uuid4()

    MARA.loc[mara_last_index, 'MANDT'] = values.mandt
    MARA.loc[mara_last_index, 'MATNR'] = rand_matnr

    MATERIALS.append(Material(matnr=rand_matnr, price=v['price']))

In [6]:
class SalesOrder:
    def __init__(self, ernam, erdat, vbtyp, customer: Customer, agreed_delivery_time: datetime) -> None:
        self.mandt = values.mandt
        self.vbeln = uuid.uuid4()
        self.erdat = erdat
        self.ernam = ernam
        self.vbtyp = vbtyp
        self.netwr = 0
        self.customer = customer
        self.delco = agreed_delivery_time

        self.vbaps = []
        self.likp_id = uuid.uuid4()
        self.mkpf_mblnr = uuid.uuid4()
        self.vbrk_id = uuid.uuid4()
        self.bkpf_id = uuid.uuid4()

        self.activity_create_sales_order()

    def activity_create_sales_order(self):
        vbak_last_index = len(VBAK)
        VBAK.loc[vbak_last_index, 'MANDT'] = self.mandt
        VBAK.loc[vbak_last_index, 'VBELN'] = self.vbeln
        VBAK.loc[vbak_last_index, 'ERNAM'] = self.ernam
        VBAK.loc[vbak_last_index, 'ERDAT'] = self.erdat
        VBAK.loc[vbak_last_index, 'VBTYP'] = 'C'
        VBAK.loc[vbak_last_index, 'KUNNR'] = self.customer
    
    def activity_create_sales_order_item(self, materials: list):
        for i, mat in enumerate(materials):
            quantity = random.randint(25, 150)
            vbap_last_index = len(VBAP)

            VBAP.loc[vbap_last_index, 'MANDT'] = self.mandt
            VBAP.loc[vbap_last_index, 'VBELN'] = self.vbeln
            VBAP.loc[vbap_last_index, 'POSNR'] = i
            VBAP.loc[vbap_last_index, 'MATNR'] = mat.matnr
            VBAP.loc[vbap_last_index, 'KWMENG'] = quantity # Cumulative Order Quantity in Sales Units
            VBAP.loc[vbap_last_index, 'NETWR'] = mat.price * 12 * 12 * quantity # calculated as a box which usually contains 12 dozens
            VBAP.loc[vbap_last_index, 'VBELN'] = self.vbeln

            self.netwr += mat.price * 12 * 12 * quantity
            self.vbaps.append(SalesOrderItem(vbeln=self.vbeln, posnr=i))

    def activity_generate_delivery_doc(self, delivery_doc_created_at: datetime, activity_by: User):        
        # generate the delivery document
        likp_last_index = len(LIKP)
        
        LIKP.loc[likp_last_index, 'MANDT'] = self.mandt
        LIKP.loc[likp_last_index, 'VBELN'] = self.likp_id
        LIKP.loc[likp_last_index, 'ERDAT'] = delivery_doc_created_at
        LIKP.loc[likp_last_index, 'ERNAM'] = activity_by

        # record LIPS
        for vbap in self.vbaps:
            lips_last_index = len(LIPS)

            LIPS.loc[lips_last_index, 'MANDT'] = self.mandt
            LIPS.loc[lips_last_index, 'VBELN'] = self.likp_id
            LIPS.loc[lips_last_index, 'POSNR'] = vbap.posnr
            LIPS.loc[lips_last_index, 'KDAUF'] = self.vbeln
            LIPS.loc[lips_last_index, 'KDPOS'] = vbap.posnr

    def activity_release_delivery(self, delivery_released_at: datetime, activity_by: User):
        # https://leanx.eu/en/sap/table/lips.html
        rand_change_nr = uuid.uuid4()
        
        # TODO include VBLB table for release type in conneciton with VBAK, VBAP, VBUK, VBUP
        # record LIPS
        for lips_index in LIPS[LIPS['VBELN'] == self.likp_id].index.values:
            cdpos_last_index = len(CDPOS)
            value_old = LIPS.loc[lips_index, 'ABART']
            LIPS.loc[lips_index, 'ABART'] = 6

            # record change CDPOS
            CDPOS.loc[cdpos_last_index, 'MANDANT'] = self.mandt
            CDPOS.loc[cdpos_last_index, 'OBJECTCLAS'] = "LIPS"
            CDPOS.loc[cdpos_last_index, 'OBJECTID'] = f"{self.mandt}{LIPS.loc[lips_index, 'VBELN']}{LIPS.loc[lips_index, 'POSNR']}"
            CDPOS.loc[cdpos_last_index, 'CHANGENR'] = rand_change_nr
            CDPOS.loc[cdpos_last_index, 'TABNAME'] = "LIPS"
            CDPOS.loc[cdpos_last_index, 'TABKEY'] = f"{self.mandt}{LIPS.loc[lips_index, 'VBELN']}"
            CDPOS.loc[cdpos_last_index, 'FNAME'] = 'ABART'
            CDPOS.loc[cdpos_last_index, 'CHNGIND'] ='U'
            CDPOS.loc[cdpos_last_index, 'VALUE_OLD'] = value_old
            CDPOS.loc[cdpos_last_index, 'VALUE_NEW'] = LIPS.loc[lips_index, 'ABART']

        # record change CDHDR
        cdhdr_last_index = len(CDHDR)
        CDHDR.loc[cdhdr_last_index, 'MANDANT'] = self.mandt
        CDHDR.loc[cdhdr_last_index, 'OBJECTCLAS'] = "LIPS"
        CDHDR.loc[cdhdr_last_index, 'OBJECTID'] = self.likp_id
        CDHDR.loc[cdhdr_last_index, 'CHANGENR'] = rand_change_nr
        CDHDR.loc[cdhdr_last_index, 'USERNAME'] = activity_by
        CDHDR.loc[cdhdr_last_index, 'UDATE'] = delivery_released_at

    def activity_ship_goods(self, shipped_at: datetime, activity_by: User):
        # record new MKPF
        mkpf_last_index = len(MKPF)

        MKPF.loc[mkpf_last_index, 'MANDT'] = self.mandt
        MKPF.loc[mkpf_last_index, 'MBLNR'] = self.mkpf_mblnr
        MKPF.loc[mkpf_last_index, 'SPE_BUDAT_UHR'] = shipped_at
        MKPF.loc[mkpf_last_index, 'USNAM'] = activity_by
        MKPF.loc[mkpf_last_index, 'USNAM'] = activity_by

        # record new MSEG
        for vbap in self.vbaps:
            mseg_last_index = len(MSEG)

            MSEG.loc[mseg_last_index, 'MANDT'] = self.mandt
            MSEG.loc[mseg_last_index, 'MBLNR'] = self.mkpf_mblnr
            MSEG.loc[mseg_last_index, 'ZEILE'] = vbap.posnr
            MSEG.loc[mseg_last_index, 'KDAUF'] = self.vbeln
            MSEG.loc[mseg_last_index, 'KDPOS'] = vbap.posnr
    
    def activity_create_billing_document(self, invoice_sent_at: datetime, activity_by: User):
        # record new billing document (VBRK)
        vbrk_last_index = len(VBRK)
        VBRK.loc[vbrk_last_index, 'MANDT'] = self.mandt
        VBRK.loc[vbrk_last_index, 'VBELN'] = self.vbrk_id
        VBRK.loc[vbrk_last_index, 'ERNAM'] = activity_by
        VBRK.loc[vbrk_last_index, 'ERDAT'] = invoice_sent_at

        # create VBRP
        for vbap in self.vbaps:
            vbrp_last_index = len(VBRP)
            VBRP.loc[vbrp_last_index, 'MANDT'] = self.mandt
            VBRP.loc[vbrp_last_index, 'VBELN'] = self.vbrk_id
            VBRP.loc[vbrp_last_index, 'POSNR'] = vbap.posnr
            VBRP.loc[vbrp_last_index, 'AUBEL'] = self.vbeln
            VBRP.loc[vbrp_last_index, 'AUPOS'] = vbap.posnr

        # record new invoice (BKPF)  ---Create Invoice---
        bkpf_last_index = len(BKPF)
        BKPF.loc[bkpf_last_index, 'MANDT'] = self.mandt
        BKPF.loc[bkpf_last_index, 'BUKRS'] = self.customer
        BKPF.loc[bkpf_last_index, 'BELNR'] = self.bkpf_id
        BKPF.loc[bkpf_last_index, 'BLDAT'] = invoice_sent_at
        BKPF.loc[bkpf_last_index, 'USNAM'] = activity_by
        BKPF.loc[bkpf_last_index, 'AWKEY'] = self.vbrk_id
        BKPF.loc[bkpf_last_index, 'AWTYP'] = 'VBRK'

        # record invoice item (BSEG)
        for vbap in self.vbaps:
            bseg_last_index = len(BSEG)
            BSEG.loc[bseg_last_index, 'MANDT'] = self.mandt
            BSEG.loc[bseg_last_index, 'BUKRS'] = self.customer
            BSEG.loc[bseg_last_index, 'BELNR'] = self.bkpf_id
            BSEG.loc[bseg_last_index, 'VBEL2'] = self.vbeln
            BSEG.loc[bseg_last_index, 'POSN2'] = vbap.posnr
            BSEG.loc[bseg_last_index, 'BUZEI'] = vbap.posnr

    def activity_receive_delivery_confirmation(self, actual_delivery_at: datetime, delivery_confirmation_received_at: datetime, activity_by: User):
        # update delivery document header
        likp_index = LIKP[LIKP['VBELN'] == self.likp_id].index.values[0]
        
        value_old = LIKP.loc[likp_index, 'SPE_ACC_APP_STS']
        LIKP.loc[likp_index, 'SPE_ACC_APP_STS'] = 'C'
        LIKP.loc[likp_index, 'LFUHR'] = actual_delivery_at

        rand_change_nr = uuid.uuid4()
        cdpos_last_index = len(CDPOS)

        # record change CDPOS
        CDPOS.loc[cdpos_last_index, 'MANDANT'] = self.mandt
        CDPOS.loc[cdpos_last_index, 'OBJECTCLAS'] = "LIKP"
        CDPOS.loc[cdpos_last_index, 'OBJECTID'] = f"{self.mandt}{self.likp_id}"
        CDPOS.loc[cdpos_last_index, 'CHANGENR'] = rand_change_nr
        CDPOS.loc[cdpos_last_index, 'TABNAME'] = "LIKP"
        CDPOS.loc[cdpos_last_index, 'TABKEY'] = f"{self.mandt}{self.likp_id}"
        CDPOS.loc[cdpos_last_index, 'FNAME'] = 'SPE_ACC_APP_STS'
        CDPOS.loc[cdpos_last_index, 'CHNGIND'] ='U'
        CDPOS.loc[cdpos_last_index, 'VALUE_OLD'] = value_old
        CDPOS.loc[cdpos_last_index, 'VALUE_NEW'] = LIKP.loc[likp_index, 'SPE_ACC_APP_STS']

        # record change CDHDR
        cdhdr_last_index = len(CDHDR)
        CDHDR.loc[cdhdr_last_index, 'MANDANT'] = self.mandt
        CDHDR.loc[cdhdr_last_index, 'OBJECTCLAS'] = "LIKP"
        CDHDR.loc[cdhdr_last_index, 'OBJECTID'] = self.likp_id
        CDHDR.loc[cdhdr_last_index, 'CHANGENR'] = rand_change_nr
        CDHDR.loc[cdhdr_last_index, 'USERNAME'] = activity_by
        CDHDR.loc[cdhdr_last_index, 'UDATE'] = delivery_confirmation_received_at

    def activity_clear_invoice(self, inovice_cleared_at: datetime):
        bseg_indices = BSEG[BSEG['BELNR'] == self.bkpf_id].index.values
        for bseg_index in bseg_indices:
            BSEG.loc[bseg_index, 'AUGDT'] = inovice_cleared_at
    
    # Deveiations
    # -----------
    def activity_set_billing_block(self, reason_for_billing_block, billing_block_set_at: datetime, activity_by: User):
        vbak_indices = VBAK[VBAK['VBELN'] == self.vbeln].index.values # FIXME assert there is only one VBAK with such VBELN, currently works for all such VBAKs
        
        # change VBAK
        for vbak_index in vbak_indices:
            value_old = VBAK.loc[vbak_index, 'FAKSK']
            VBAK.loc[vbak_index, 'FAKSK'] = reason_for_billing_block # TODO currently on VBAK level, check if it needs to be in VBAP level as well
            rand_change_nr = uuid.uuid4()

            # record change CDPOS
            cdpos_last_index = len(CDPOS)
            CDPOS.loc[cdpos_last_index, 'MANDANT'] = self.mandt
            CDPOS.loc[cdpos_last_index, 'OBJECTCLAS'] = "VBAK"
            CDPOS.loc[cdpos_last_index, 'OBJECTID'] = f"{self.mandt}{self.vbeln}"
            CDPOS.loc[cdpos_last_index, 'CHANGENR'] = rand_change_nr
            CDPOS.loc[cdpos_last_index, 'TABNAME'] = "VBAK"
            CDPOS.loc[cdpos_last_index, 'TABKEY'] = f"{self.mandt}{self.vbeln}"
            CDPOS.loc[cdpos_last_index, 'FNAME'] = 'FAKSK'
            CDPOS.loc[cdpos_last_index, 'CHNGIND'] ='U'
            CDPOS.loc[cdpos_last_index, 'VALUE_OLD'] = value_old
            CDPOS.loc[cdpos_last_index, 'VALUE_NEW'] = VBAK.loc[vbak_index, 'FAKSK']

            # record change CDHDR
            cdhdr_last_index = len(CDHDR)
            CDHDR.loc[cdhdr_last_index, 'MANDANT'] = self.mandt
            CDHDR.loc[cdhdr_last_index, 'OBJECTCLAS'] = "VBAK"
            CDHDR.loc[cdhdr_last_index, 'OBJECTID'] = self.vbeln
            CDHDR.loc[cdhdr_last_index, 'CHANGENR'] = rand_change_nr
            CDHDR.loc[cdhdr_last_index, 'USERNAME'] = activity_by
            CDHDR.loc[cdhdr_last_index, 'UDATE'] = billing_block_set_at

    def activity_remove_billing_block(self, billing_block_removed_at: datetime, activity_by: User):
        vbak_indices = VBAK[VBAK['VBELN'] == self.vbeln].index.values # FIXME assert there is only one VBAK with such VBELN, currently works for all such VBAKs

        # change VBAK
        for vbak_index in vbak_indices:
            value_old = VBAK.loc[vbak_index, 'FAKSK']
            VBAK.loc[vbak_index, 'FAKSK'] = pd.NA # TODO currently on VBAK level, check if it needs to be in VBAP level as well
            rand_change_nr = uuid.uuid4()

            # record change CDPOS
            cdpos_last_index = len(CDPOS)
            CDPOS.loc[cdpos_last_index, 'MANDANT'] = self.mandt
            CDPOS.loc[cdpos_last_index, 'OBJECTCLAS'] = "VBAK"
            CDPOS.loc[cdpos_last_index, 'OBJECTID'] = f"{self.mandt}{self.vbeln}"
            CDPOS.loc[cdpos_last_index, 'CHANGENR'] = rand_change_nr
            CDPOS.loc[cdpos_last_index, 'TABNAME'] = "VBAK"
            CDPOS.loc[cdpos_last_index, 'TABKEY'] = f"{self.mandt}{self.vbeln}"
            CDPOS.loc[cdpos_last_index, 'FNAME'] = 'FAKSK'
            CDPOS.loc[cdpos_last_index, 'CHNGIND'] ='U'
            CDPOS.loc[cdpos_last_index, 'VALUE_OLD'] = value_old
            CDPOS.loc[cdpos_last_index, 'VALUE_NEW'] = VBAK.loc[vbak_index, 'FAKSK']

            # record change CDHDR
            cdhdr_last_index = len(CDHDR)
            CDHDR.loc[cdhdr_last_index, 'MANDANT'] = self.mandt
            CDHDR.loc[cdhdr_last_index, 'OBJECTCLAS'] = "VBAK"
            CDHDR.loc[cdhdr_last_index, 'OBJECTID'] = self.vbeln
            CDHDR.loc[cdhdr_last_index, 'CHANGENR'] = rand_change_nr
            CDHDR.loc[cdhdr_last_index, 'USERNAME'] = activity_by
            CDHDR.loc[cdhdr_last_index, 'UDATE'] = billing_block_removed_at

    def activity_set_delivery_block(self, reason_for_delivery_block, delivery_block_set_at: datetime, activity_by: User):
        vbak_indices = VBAK[VBAK['VBELN'] == self.vbeln].index.values # FIXME assert there is only one VBAK with such VBELN, currently works for all such VBAKs

        # change VBAK
        for vbak_index in vbak_indices:
            value_old = VBAK.loc[vbak_index, 'LIFSK']
            VBAK.loc[vbak_index, 'LIFSK'] = reason_for_delivery_block # TODO currently on VBAK level, check if it needs to be in VBAP level as well
            rand_change_nr = uuid.uuid4()

            # record change CDPOS
            cdpos_last_index = len(CDPOS)
            CDPOS.loc[cdpos_last_index, 'MANDANT'] = self.mandt
            CDPOS.loc[cdpos_last_index, 'OBJECTCLAS'] = "VBAK"
            CDPOS.loc[cdpos_last_index, 'OBJECTID'] = f"{self.mandt}{self.vbeln}"
            CDPOS.loc[cdpos_last_index, 'CHANGENR'] = rand_change_nr
            CDPOS.loc[cdpos_last_index, 'TABNAME'] = "VBAK"
            CDPOS.loc[cdpos_last_index, 'TABKEY'] = f"{self.mandt}{self.vbeln}"
            CDPOS.loc[cdpos_last_index, 'FNAME'] = 'LIFSK'
            CDPOS.loc[cdpos_last_index, 'CHNGIND'] ='U'
            CDPOS.loc[cdpos_last_index, 'VALUE_OLD'] = value_old
            CDPOS.loc[cdpos_last_index, 'VALUE_NEW'] = VBAK.loc[vbak_index, 'LIFSK']

            # record change CDHDR
            cdhdr_last_index = len(CDHDR)
            CDHDR.loc[cdhdr_last_index, 'MANDANT'] = self.mandt
            CDHDR.loc[cdhdr_last_index, 'OBJECTCLAS'] = "VBAK"
            CDHDR.loc[cdhdr_last_index, 'OBJECTID'] = self.vbeln
            CDHDR.loc[cdhdr_last_index, 'CHANGENR'] = rand_change_nr
            CDHDR.loc[cdhdr_last_index, 'USERNAME'] = activity_by
            CDHDR.loc[cdhdr_last_index, 'UDATE'] = delivery_block_set_at

    def activity_remove_delivery_block(self, delivery_block_removed_at: datetime, activity_by: User):
        vbak_indices = VBAK[VBAK['VBELN'] == self.vbeln].index.values # FIXME assert there is only one VBAK with such VBELN, currently works for all such VBAKs

        # change VBAK
        for vbak_index in vbak_indices:
            value_old = VBAK.loc[vbak_index, 'LIFSK']
            VBAK.loc[vbak_index, 'LIFSK'] = pd.NA # TODO currently on VBAK level, check if it needs to be in VBAP level as well
            rand_change_nr = uuid.uuid4()

            # record change CDPOS
            cdpos_last_index = len(CDPOS)
            CDPOS.loc[cdpos_last_index, 'MANDANT'] = self.mandt
            CDPOS.loc[cdpos_last_index, 'OBJECTCLAS'] = "VBAK"
            CDPOS.loc[cdpos_last_index, 'OBJECTID'] = f"{self.mandt}{self.vbeln}"
            CDPOS.loc[cdpos_last_index, 'CHANGENR'] = rand_change_nr
            CDPOS.loc[cdpos_last_index, 'TABNAME'] = "VBAK"
            CDPOS.loc[cdpos_last_index, 'TABKEY'] = f"{self.mandt}{self.vbeln}"
            CDPOS.loc[cdpos_last_index, 'FNAME'] = 'LIFSK'
            CDPOS.loc[cdpos_last_index, 'CHNGIND'] ='U'
            CDPOS.loc[cdpos_last_index, 'VALUE_OLD'] = value_old
            CDPOS.loc[cdpos_last_index, 'VALUE_NEW'] = VBAK.loc[vbak_index, 'LIFSK']

            # record change CDHDR
            cdhdr_last_index = len(CDHDR)
            CDHDR.loc[cdhdr_last_index, 'MANDANT'] = self.mandt
            CDHDR.loc[cdhdr_last_index, 'OBJECTCLAS'] = "VBAK"
            CDHDR.loc[cdhdr_last_index, 'OBJECTID'] = self.vbeln
            CDHDR.loc[cdhdr_last_index, 'CHANGENR'] = rand_change_nr
            CDHDR.loc[cdhdr_last_index, 'USERNAME'] = activity_by
            CDHDR.loc[cdhdr_last_index, 'UDATE'] = delivery_block_removed_at

    def activity_return_order(self, order_returned_at: datetime, activity_by: User): # TODO add retrun by order item instead of order doc
        vbak_indices = VBAK[VBAK['VBELN'] == self.vbeln].index.values # FIXME assert there is only one VBAK with such VBELN, currently works for all such VBAKs

        # change VBAK
        for vbak_index in vbak_indices:
            value_old = VBAK.loc[vbak_index, 'VBTYP']
            VBAK.loc[vbak_index, 'VBTYP'] = 'H' # TODO currently on VBAK level, check if it needs to be in VBAP level as well
            rand_change_nr = uuid.uuid4()

            # record change CDPOS
            cdpos_last_index = len(CDPOS)
            CDPOS.loc[cdpos_last_index, 'MANDANT'] = self.mandt
            CDPOS.loc[cdpos_last_index, 'OBJECTCLAS'] = "VBAK"
            CDPOS.loc[cdpos_last_index, 'OBJECTID'] = f"{self.mandt}{self.vbeln}"
            CDPOS.loc[cdpos_last_index, 'CHANGENR'] = rand_change_nr
            CDPOS.loc[cdpos_last_index, 'TABNAME'] = "VBAK"
            CDPOS.loc[cdpos_last_index, 'TABKEY'] = f"{self.mandt}{self.vbeln}"
            CDPOS.loc[cdpos_last_index, 'FNAME'] = 'VBTYP'
            CDPOS.loc[cdpos_last_index, 'CHNGIND'] ='U'
            CDPOS.loc[cdpos_last_index, 'VALUE_OLD'] = value_old
            CDPOS.loc[cdpos_last_index, 'VALUE_NEW'] = VBAK.loc[vbak_index, 'VBTYP']

            # record change CDHDR
            cdhdr_last_index = len(CDHDR)
            CDHDR.loc[cdhdr_last_index, 'MANDANT'] = self.mandt
            CDHDR.loc[cdhdr_last_index, 'OBJECTCLAS'] = "VBAK"
            CDHDR.loc[cdhdr_last_index, 'OBJECTID'] = self.vbeln
            CDHDR.loc[cdhdr_last_index, 'CHANGENR'] = rand_change_nr
            CDHDR.loc[cdhdr_last_index, 'USERNAME'] = activity_by
            CDHDR.loc[cdhdr_last_index, 'UDATE'] = order_returned_at

    def activity_cancel_order(self, order_cancelled_at: datetime, activity_by: User):
        vbak_indices = VBAK[VBAK['VBELN'] == self.vbeln].index.values # FIXME assert there is only one VBAK with such VBELN, currently works for all such VBAKs

        # change VBAK
        for vbak_index in vbak_indices:
            value_old = VBAK.loc[vbak_index, 'VBTYP']
            VBAK.loc[vbak_index, 'VBTYP'] = 'CANCELLED' # TODO currently on VBAK level, check if it needs to be in VBAP level as well
            rand_change_nr = uuid.uuid4()

            # record change CDPOS
            cdpos_last_index = len(CDPOS)
            CDPOS.loc[cdpos_last_index, 'MANDANT'] = self.mandt
            CDPOS.loc[cdpos_last_index, 'OBJECTCLAS'] = "VBAK"
            CDPOS.loc[cdpos_last_index, 'OBJECTID'] = f"{self.mandt}{self.vbeln}"
            CDPOS.loc[cdpos_last_index, 'CHANGENR'] = rand_change_nr
            CDPOS.loc[cdpos_last_index, 'TABNAME'] = "VBAK"
            CDPOS.loc[cdpos_last_index, 'TABKEY'] = f"{self.mandt}{self.vbeln}"
            CDPOS.loc[cdpos_last_index, 'FNAME'] = 'VBTYP'
            CDPOS.loc[cdpos_last_index, 'CHNGIND'] ='U'
            CDPOS.loc[cdpos_last_index, 'VALUE_OLD'] = value_old
            CDPOS.loc[cdpos_last_index, 'VALUE_NEW'] = VBAK.loc[vbak_index, 'VBTYP']

            # record change CDHDR
            cdhdr_last_index = len(CDHDR)
            CDHDR.loc[cdhdr_last_index, 'MANDANT'] = self.mandt
            CDHDR.loc[cdhdr_last_index, 'OBJECTCLAS'] = "VBAK"
            CDHDR.loc[cdhdr_last_index, 'OBJECTID'] = self.vbeln
            CDHDR.loc[cdhdr_last_index, 'CHANGENR'] = rand_change_nr
            CDHDR.loc[cdhdr_last_index, 'USERNAME'] = activity_by
            CDHDR.loc[cdhdr_last_index, 'UDATE'] = order_cancelled_at
        

In [7]:
for i in range(3):
    so_creator = USERS[0].bname
    latest_time = helpers.generate_random_datetime(start_date=datetime(2009, 1, 1), end_date=datetime(2010, 1, 1))
    shipping_condition = 1
    customer = CUSTOMERS[0].kunnr
    agreed_delivery_time = latest_time + helpers.UPTO_YEAR()

    sales_order = SalesOrder(ernam=so_creator, erdat=latest_time, vbtyp=shipping_condition, customer=customer, agreed_delivery_time=agreed_delivery_time)
    materials = MATERIALS[-3:]
    sales_order.activity_create_sales_order_item(materials=materials)

    delivery_doc_created_by = USERS[0].bname
    latest_time += helpers.UPTO_WEEK()
    sales_order.activity_generate_delivery_doc(delivery_doc_created_at=latest_time, activity_by=delivery_doc_created_by)

    delivery_block_set_by = USERS[0].bname
    latest_time += helpers.UPTO_WEEK()
    sales_order.activity_set_delivery_block(reason_for_delivery_block=1, delivery_block_set_at=latest_time, activity_by=delivery_block_set_by)

    delivery_block_removed_by = USERS[0].bname
    latest_time += helpers.UPTO_WEEK()
    sales_order.activity_remove_delivery_block(delivery_block_removed_at=latest_time, activity_by=delivery_block_removed_by)

    delivery_released_by = USERS[0].bname
    latest_time += helpers.UPTO_WEEK()
    sales_order.activity_release_delivery(delivery_released_at=latest_time, activity_by=delivery_released_by)

    # deviation
    billing_block_set_by = USERS[0].bname
    latest_time += helpers.UPTO_WEEK()
    sales_order.activity_set_billing_block(reason_for_billing_block=1, billing_block_set_at=latest_time, activity_by=billing_block_set_by)

    shipped_by = USERS[0].bname
    latest_time += helpers.UPTO_WEEK()
    sales_order.activity_ship_goods(shipped_at=latest_time, activity_by=shipped_by)

    # deviation
    billing_block_removed_by = USERS[0].bname
    latest_time += helpers.UPTO_WEEK()
    sales_order.activity_remove_billing_block(billing_block_removed_at=latest_time, activity_by=billing_block_removed_by)

    invoice_sent_by = USERS[0].bname
    latest_time += helpers.UPTO_WEEK()
    sales_order.activity_create_billing_document(invoice_sent_at=latest_time, activity_by=invoice_sent_by)

    delivery_received_by = USERS[0].bname
    latest_time += helpers.UPTO_WEEK()
    sales_order.activity_receive_delivery_confirmation(actual_delivery_at=latest_time, delivery_confirmation_received_at=latest_time, activity_by=delivery_received_by)

    invoice_cleared_by = USERS[0].bname
    latest_time += helpers.UPTO_WEEK()
    sales_order.activity_clear_invoice(inovice_cleared_at=latest_time)
    
    # deviation
    order_returned_by = USERS[0].bname
    latest_time += helpers.UPTO_WEEK()
    sales_order.activity_return_order(order_returned_at=latest_time, activity_by=order_returned_by)
    
    # deviation
    order_cancelled_by = USERS[0].bname
    latest_time += helpers.UPTO_WEEK()
    sales_order.activity_cancel_order(order_cancelled_at=latest_time, activity_by=order_cancelled_by)

In [8]:
version = str(datetime.now())
os.mkdir(f'data/{version}')

VBAK.dropna(axis=1, how='all', inplace=True)
VBAK.to_csv(f'data/{version}/VBAK.csv', index=False)

VBAP.dropna(axis=1, how='all', inplace=True)
VBAP.to_csv(f'data/{version}/VBAP.csv', index=False)


LIKP.dropna(axis=1, how='all', inplace=True)
LIKP.to_csv(f'data/{version}/LIKP.csv', index=False)

LIPS.dropna(axis=1, how='all', inplace=True)
LIPS.to_csv(f'data/{version}/LIPS.csv', index=False)


MKPF.dropna(axis=1, how='all', inplace=True)
MKPF.to_csv(f'data/{version}/MKPF.csv', index=False)

MSEG.dropna(axis=1, how='all', inplace=True)
MSEG.to_csv(f'data/{version}/MSEG.csv', index=False)


BKPF.dropna(axis=1, how='all', inplace=True)
BKPF.to_csv(f'data/{version}/BKPF.csv', index=False)

BSEG.dropna(axis=1, how='all', inplace=True)
BSEG.to_csv(f'data/{version}/BSEG.csv', index=False)


VBRK.dropna(axis=1, how='all', inplace=True)
VBRK.to_csv(f'data/{version}/VBRK.csv', index=False)

VBRP.dropna(axis=1, how='all', inplace=True)
VBRP.to_csv(f'data/{version}/VBRP.csv', index=False)


CDHDR.dropna(axis=1, how='all', inplace=True)
CDHDR.to_csv(f'data/{version}/CDHDR.csv', index=False)

CDPOS.dropna(axis=1, how='all', inplace=True)
CDPOS.to_csv(f'data/{version}/CDPOS.csv', index=False)


USR02.dropna(axis=1, how='all', inplace=True)
USR02.to_csv(f'data/{version}/USR02.csv', index=False)

MARA.dropna(axis=1, how='all', inplace=True)
MARA.to_csv(f'data/{version}/MARA.csv', index=False)

KNA1.dropna(axis=1, how='all', inplace=True)
KNA1.to_csv(f'data/{version}/KNA1.csv', index=False)
